In [14]:
!pip install imblearn
!pip install xgboost
!pip install catboost
!pip install lightgbm
!pip install optuna
!pip install nltk
!pip install gensim

^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/opt/conda/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 177, in handle_pip_version_check
    session = self._build_session(
              ^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 122, in _build_session
    session = PipSession(
              ^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pip/

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

import pandas as pd

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import optuna

from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_param_importances
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import make_scorer
from imblearn.ensemble import BalancedBaggingClassifier
import joblib
from sklearn.metrics import matthews_corrcoef

In [98]:
import warnings

warnings.filterwarnings('ignore')

In [14]:
from src.functions.vectorize_functions import vectorize_tfidf, vectorize_glove, vectorize_w2v

#### Evaluations-Funktionen

In [25]:
def simple_evaluation(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [101]:
evaluation = pd.DataFrame(
    columns=["model", "train_acc", "train_prec", "train_rec", "train_f1", "train_mcc", 
             "test_acc", "test_prec","test_rec", "test_f1", "test_mcc"])


def add_to_eval_df(model, model_name, x_data_train, y_data_train, x_data_test, y_data_test):
    train_pred = model.predict(x_data_train)
    train_acc = accuracy_score(y_data_train, train_pred)
    train_precision = precision_score(y_data_train, train_pred)
    train_recall = recall_score(y_data_train, train_pred)
    train_f1 = f1_score(y_data_train, train_pred)
    train_mcc = matthews_corrcoef(y_data_train, train_pred)

    test_pred = model.predict(x_data_test)
    test_acc = accuracy_score(y_data_test, test_pred)
    test_precision = precision_score(y_data_test, test_pred)
    test_recall = recall_score(y_data_test, test_pred)
    test_f1 = f1_score(y_data_test, test_pred)
    test_mcc = matthews_corrcoef(y_data_test, test_pred)

    evaluation.loc[len(evaluation.index)] = [model_name, 
                                             train_acc, train_precision, train_recall,train_f1,train_mcc,
                                             test_acc, test_precision, test_recall, test_f1, test_mcc]

In [164]:
evaluation_tuning = pd.DataFrame(
    columns=["model", "tuning", "scorer", "best_params",
             "train_acc", "train_prec", "train_rec", "train_f1", "train_mcc",
             "test_acc", "test_prec", "test_rec", "test_f1", "test_mcc",
             "clf_rep_train", "clf_rep_test"])


def add_to_tuning_eval_df(model, model_name, tuning, scorer, best_params, x_data_train, y_data_train, x_data_test,
                          y_data_test):
    train_pred = model.predict(x_data_train)
    train_acc = accuracy_score(y_data_train, train_pred)
    train_precision = precision_score(y_data_train, train_pred)
    train_recall = recall_score(y_data_train, train_pred)
    train_f1 = f1_score(y_data_train, train_pred)
    train_mcc = matthews_corrcoef(y_data_train, train_pred)
    clf_rep_train = classification_report(y_data_train, train_pred)

    test_pred = model.predict(x_data_test)
    test_acc = accuracy_score(y_data_test, test_pred)
    test_precision = precision_score(y_data_test, test_pred)
    test_recall = recall_score(y_data_test, test_pred)
    test_f1 = f1_score(y_data_test, test_pred)
    test_mcc = matthews_corrcoef(y_data_test, test_pred)
    clf_rep_test = classification_report(y_data_test, test_pred)

    evaluation_tuning.loc[len(evaluation_tuning.index)] = [model_name, tuning, scorer, best_params,
                                                           train_acc, train_precision, train_recall, train_f1,train_mcc,
                                                           test_acc, test_precision, test_recall, test_f1,test_mcc,
                                                           clf_rep_train, clf_rep_test]

#### Daten einlesen

In [11]:
df_train = pd.read_csv("../../../data/twitter_hate-speech/train_cleaned.csv", index_col=0)
df_train = df_train[df_train.tweet.notna() & df_train.tweet_cleaned.notna()]
df_train.head()

,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
id,,,,,,
8886,0,@user #cinemaaawards final rehearsals!! geari...,cinema award final rehearsal gear evening butt...,1,"['#cinemaaawards', '#butterflies', '#stage']",NaN
909,0,istg this is the best cheese ta but dayum expe...,tg good cheese day um expensive,0,[],NaN
27613,0,this was amazing. the weather was not. #musica...,amazing weather musical london matilda west en...,0,"['#musical', '#london', '#matilda', '#westend'...",NaN
15999,0,yes! #talented #sexy ‘criminal minds’ casts ...,yes talented sexy ' criminal mind ' cast serie...,2,"['#talented', '#sexy']",NaN
23817,0,want to be while being #successful? see how ...,want successful see work life balance help,2,"['#successful', '#worklifebalance']",NaN


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20178 entries, 8886 to 16978
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   label          20178 non-null  int64 
 1   tweet          20178 non-null  object
 2   tweet_cleaned  20178 non-null  object
 3   user_handle    20178 non-null  int64 
 4   hashtags       20178 non-null  object
 5   emojis         4305 non-null   object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


#### Vergleich von Vektorisierungsarten

##### TF-IDF

In [15]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf, tfidf_vectorizer = vectorize_tfidf(df=df_train,
                                                                                 text_column="tweet_cleaned",
                                                                                 label_column="label")

##### Word2Vec

In [ ]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v, vectorizer_w2v = vectorize_w2v(df=df_train,
                                                                 text_column="tweet_cleaned",
                                                                 label_column="label")

##### GloVe

In [ ]:
X_train_gl, X_test_gl, y_train_gl, y_test_gl, vectorizer_gl = vectorize_glove(df=df_train,
                                                               text_column="tweet_cleaned",
                                                               label_column="label")

##### Vergleich der Vektorisierungsarten mit Baseline-Modell (RandomForest)

In [ ]:
rf_base_tf = RandomForestClassifier(n_jobs=-1)
rf_base_tf.fit(X_train_tf, y_train_tf)
y_pred_base_tf = rf_base_tf.predict(X_test_tf)

In [ ]:
simple_evaluation(y_test_tf, y_pred_base_tf)

In [ ]:
rf_base_w2v = RandomForestClassifier(n_jobs=-1)
rf_base_w2v.fit(X_train_w2v, y_train_w2v)
y_pred_base_w2v = rf_base_w2v.predict(X_test_w2v)

In [ ]:
simple_evaluation(y_test_w2v, y_pred_base_w2v)

In [ ]:
rf_base_gl = RandomForestClassifier(n_jobs=-1)
rf_base_gl.fit(X_train_gl, y_train_gl)
y_pred_base_gl = rf_base_gl.predict(X_test_gl)

In [ ]:
simple_evaluation(y_test_gl, y_pred_base_gl)

###### Entscheidung: TF-IDF
==> W2V und GLOVE sind beide schlechter als TF-IDF bei F1 und Recall des Targets

W2V und GLOVE werden nicht mehr betrachtet

In [18]:
X_train, X_test, y_train, y_test = X_train_tf, X_test_tf, y_train_tf, y_test_tf

In [19]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (14124, 15658)
y_train (14124,)
X_test (6054, 15658)
y_test (6054,)


#### Vergleich von Modellen ohne Hyperparamtertuning

##### RandomForestClassifier

In [103]:
rf_base = RandomForestClassifier(n_jobs=-1)
rf_base.fit(X_train, y_train)

add_to_eval_df(model=rf_base, model_name="RandomForestClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

##### BalancedRandomForestClassifier / BalancedBaggingClassifier (imblearn)

In [104]:
brf_base = BalancedRandomForestClassifier(n_jobs=-1)
brf_base.fit(X_train, y_train)

add_to_eval_df(model=brf_base, model_name="BalancedRandomForestClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

In [105]:
brf_img_base = BalancedBaggingClassifier(n_jobs=-1)
brf_img_base.fit(X_train, y_train)

add_to_eval_df(model=brf_img_base, model_name="BalancedBaggingClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

##### XGBClassifier

In [106]:
xgb_base = XGBClassifier(n_jobs=-1)
xgb_base.fit(X_train, y_train)

add_to_eval_df(model=xgb_base, model_name="XGBClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

##### CatBoostClassifier

In [107]:
cat_base = CatBoostClassifier()
cat_base.fit(X_train, y_train)

add_to_eval_df(model=cat_base, model_name="CatBoostClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

Learning rate set to 0.030812
0:	learn: 0.6565849	total: 47ms	remaining: 47s
1:	learn: 0.6213407	total: 83.1ms	remaining: 41.5s
2:	learn: 0.5901405	total: 119ms	remaining: 39.4s
3:	learn: 0.5612668	total: 153ms	remaining: 38.2s
4:	learn: 0.5351362	total: 188ms	remaining: 37.5s
5:	learn: 0.5101657	total: 223ms	remaining: 37s
6:	learn: 0.4881079	total: 257ms	remaining: 36.4s
7:	learn: 0.4670665	total: 290ms	remaining: 36s
8:	learn: 0.4473869	total: 324ms	remaining: 35.6s
9:	learn: 0.4295148	total: 359ms	remaining: 35.5s
10:	learn: 0.4122485	total: 392ms	remaining: 35.3s
11:	learn: 0.3974217	total: 425ms	remaining: 35s
12:	learn: 0.3828064	total: 459ms	remaining: 34.9s
13:	learn: 0.3694903	total: 493ms	remaining: 34.7s
14:	learn: 0.3576871	total: 529ms	remaining: 34.7s
15:	learn: 0.3462710	total: 564ms	remaining: 34.7s
16:	learn: 0.3362762	total: 599ms	remaining: 34.6s
17:	learn: 0.3252127	total: 635ms	remaining: 34.7s
18:	learn: 0.3165478	total: 672ms	remaining: 34.7s
19:	learn: 0.307171

##### LGBMClassifier

In [108]:
lgb_base = LGBMClassifier(n_jobs=-1)
lgb_base.fit(X_train, y_train)

add_to_eval_df(model=lgb_base, model_name="LGBMClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

[LightGBM] [Info] Number of positive: 792, number of negative: 12217
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.531807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25724
[LightGBM] [Info] Number of data points in the train set: 13009, number of used features: 981
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.060881 -> initscore=-2.736022
[LightGBM] [Info] Start training from score -2.736022


##### StackingClassifier

In [109]:
base_models_stack = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', LinearSVC(random_state=42))
]
stack_base = StackingClassifier(estimators=base_models_stack, n_jobs=-1)
stack_base.fit(X_train, y_train)

add_to_eval_df(model=stack_base, model_name="StackingClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

##### VotingClassifier

In [110]:
base_models_vote = [
    ('lr', LogisticRegression(random_state=42)), ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('svc', LinearSVC(random_state=42))
]

vote_base = VotingClassifier(estimators=base_models_vote, voting='hard', n_jobs=-1)
vote_base.fit(X_train, y_train)

add_to_eval_df(model=vote_base, model_name="VotingClassifier", x_data_train=X_train, y_data_train=y_train,
               x_data_test=X_test, y_data_test=y_test)

##### Vergleich der Modelle ohne Tuning

In [111]:
evaluation.sort_values(by=["test_f1"], ascending=False)

,model,train_acc,train_prec,train_rec,train_f1,train_mcc,test_acc,test_prec,test_rec,test_f1,test_mcc
6,StackingClassifier,0.999001,0.998720,0.984848,0.991736,0.991231,0.957855,0.785124,0.509383,0.617886,0.612331
1,BalancedRandomForestClassifier,0.961027,0.609700,1.000000,0.757532,0.764459,0.925036,0.458564,0.667560,0.543668,0.514930
0,RandomForestClassifier,0.999846,1.000000,0.997475,0.998736,0.998655,0.954986,0.886076,0.375335,0.527307,0.559926
5,LGBMClassifier,0.970098,0.950783,0.536616,0.686037,0.702046,0.950681,0.747475,0.396783,0.518389,0.522689
3,XGBClassifier,0.966177,0.946701,0.470960,0.629005,0.654710,0.951220,0.791908,0.367292,0.501832,0.519270
7,VotingClassifier,0.995157,0.998632,0.921717,0.958634,0.956933,0.953730,0.902098,0.345845,0.500000,0.542353
2,BalancedBaggingClassifier,0.940887,0.507672,0.960859,0.664339,0.674321,0.903336,0.366129,0.608579,0.457200,0.423628
4,CatBoostClassifier,0.962026,0.962733,0.391414,0.556553,0.600873,0.949426,0.822695,0.310992,0.451362,0.487258


###### Zwischenenergebnis

- große Unterschiede zwischen Precision und Recall => vmtl. durch Klassenungleichgewicht => Basistraining erneut mit SMOTE-Daten durchführen

#### Vergleich von Modellen ohne Hyperparamtertuning - SMOTE

In [20]:
resampler = SMOTE()
X_smote, y_smote = resampler.fit_resample(X_train, y_train)

In [21]:
print("X_smote", X_smote.shape)
print("y_smote", y_smote.shape)

X_smote (26320, 15658)
y_smote (26320,)


##### RandomForestClassifier

In [114]:
rf_base = RandomForestClassifier(n_jobs=-1)
rf_base.fit(X_smote, y_smote)

add_to_eval_df(model=rf_base, model_name="RandomForestClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

##### BalancedRandomForestClassifier / BalancedBaggingClassifier (imblearn)

In [115]:
brf_base_smote = BalancedRandomForestClassifier(n_jobs=-1)
brf_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=brf_base_smote, model_name="BalancedRandomForestClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

In [116]:
brf_base_pred_smote = brf_base_smote.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_base_pred_smote)

Accuracy: 0.9573170731707317

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      5203
           1       0.82      0.47      0.59       373

    accuracy                           0.96      5576
   macro avg       0.89      0.73      0.79      5576
weighted avg       0.95      0.96      0.95      5576



In [117]:
brf_img_base_smote = BalancedBaggingClassifier(n_jobs=-1)
brf_img_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=brf_img_base_smote, model_name="BalancedBaggingClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

##### XGBClassifier

In [118]:
xgb_base_smote = XGBClassifier(n_jobs=-1)
xgb_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=xgb_base_smote, model_name="XGBClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

##### CatBoostClassifier

In [119]:
cat_base_smote = CatBoostClassifier()
cat_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=cat_base_smote, model_name="CatBoostClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

Learning rate set to 0.040328
0:	learn: 0.6707595	total: 119ms	remaining: 1m 58s
1:	learn: 0.6503150	total: 238ms	remaining: 1m 58s
2:	learn: 0.6316783	total: 332ms	remaining: 1m 50s
3:	learn: 0.6143873	total: 445ms	remaining: 1m 50s
4:	learn: 0.6000754	total: 516ms	remaining: 1m 42s
5:	learn: 0.5855538	total: 593ms	remaining: 1m 38s
6:	learn: 0.5752644	total: 692ms	remaining: 1m 38s
7:	learn: 0.5627152	total: 794ms	remaining: 1m 38s
8:	learn: 0.5512049	total: 900ms	remaining: 1m 39s
9:	learn: 0.5416065	total: 993ms	remaining: 1m 38s
10:	learn: 0.5332116	total: 1.11s	remaining: 1m 40s
11:	learn: 0.5255628	total: 1.21s	remaining: 1m 39s
12:	learn: 0.5183177	total: 1.28s	remaining: 1m 37s
13:	learn: 0.5118141	total: 1.38s	remaining: 1m 37s
14:	learn: 0.5052488	total: 1.5s	remaining: 1m 38s
15:	learn: 0.4997013	total: 1.6s	remaining: 1m 38s
16:	learn: 0.4931755	total: 1.68s	remaining: 1m 36s
17:	learn: 0.4867453	total: 1.82s	remaining: 1m 39s
18:	learn: 0.4819798	total: 1.92s	remaining: 1

##### LGBMClassifier

In [120]:
lgb_base_smote = LGBMClassifier(n_jobs=-1)
lgb_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=lgb_base_smote, model_name="LGBMClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

[LightGBM] [Info] Number of positive: 12217, number of negative: 12217
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.166648 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 80475
[LightGBM] [Info] Number of data points in the train set: 24434, number of used features: 2666
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


##### StackingClassifier

In [121]:
base_models_stack_smote = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', LinearSVC(random_state=42))
]
stack_base_smote = StackingClassifier(estimators=base_models_stack_smote, n_jobs=-1)
stack_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=stack_base_smote, model_name="StackingClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

##### VotingClassifier

In [122]:
base_models_vote_smote = [
    ('lr', LogisticRegression(random_state=42)), ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('svc', LinearSVC(random_state=42))
]

vote_base_smote = VotingClassifier(estimators=base_models_vote_smote, voting='hard', n_jobs=-1)
vote_base_smote.fit(X_smote, y_smote)

add_to_eval_df(model=vote_base_smote, model_name="VotingClassifier-SMOTE", x_data_train=X_smote, y_data_train=y_smote,
               x_data_test=X_test, y_data_test=y_test)

##### Vergleich der Modelle ohne Tuning - SMOTE

In [123]:
evaluation.sort_values(by=["test_f1"], ascending=False).head(15)

,model,train_acc,train_prec,train_rec,train_f1,train_mcc,test_acc,test_prec,test_rec,test_f1,test_mcc
15,VotingClassifier-SMOTE,0.998117,0.996573,0.999673,0.998120,0.996240,0.952834,0.659884,0.608579,0.633194,0.608603
6,StackingClassifier,0.999001,0.998720,0.984848,0.991736,0.991231,0.957855,0.785124,0.509383,0.617886,0.612331
9,BalancedRandomForestClassifier-SMOTE,0.999918,1.000000,0.999836,0.999918,0.999836,0.957317,0.816901,0.466488,0.593857,0.598264
8,RandomForestClassifier-SMOTE,0.999918,1.000000,0.999836,0.999918,0.999836,0.956779,0.800000,0.471850,0.593592,0.594702
12,CatBoostClassifier-SMOTE,0.984898,0.986052,0.983711,0.984880,0.969799,0.947991,0.623881,0.560322,0.590395,0.563640
14,StackingClassifier-SMOTE,0.999714,0.999754,0.999673,0.999714,0.999427,0.956600,0.813397,0.455764,0.584192,0.589631
13,LGBMClassifier-SMOTE,0.979946,0.981048,0.978800,0.979923,0.959894,0.944225,0.583784,0.579088,0.581427,0.551554
1,BalancedRandomForestClassifier,0.961027,0.609700,1.000000,0.757532,0.764459,0.925036,0.458564,0.667560,0.543668,0.514930
11,XGBClassifier-SMOTE,0.974830,0.984062,0.965294,0.974588,0.949833,0.943329,0.591054,0.495979,0.539359,0.511636
10,BalancedBaggingClassifier-SMOTE,0.997299,0.998033,0.996562,0.997297,0.994599,0.935079,0.513447,0.563003,0.537084,0.502872


###### Entscheidung
- Sampling konnte erhebliche Verbesserungen erzielen

#### Hyperparametertuning ausgewählter Modelle

In [165]:
def custom_f1_macro_score(preds, labels):
    # pred_flat = np.argmax(preds, axis=1).flatten()
    pred_flat = preds.astype(int).tolist()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, average='macro')

In [166]:
def custom_f1_target_score(preds, labels):
    # pred_flat = np.argmax(preds, axis=1).flatten()
    pred_flat = preds.astype(int).tolist()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, pos_label=1)

In [167]:
def mcc(preds, labels):
    # pred_flat = np.argmax(preds, axis=1).flatten()
    pred_flat = preds.astype(int).tolist()
    labels_flat = labels.flatten()
    return matthews_corrcoef(y_true=labels_flat, y_pred=pred_flat)

In [168]:
custom_scorer_macro = make_scorer(custom_f1_macro_score)
custom_scorer_target = make_scorer(custom_f1_target_score)
custom_scorer_mcc = make_scorer(mcc)

##### Modell 1: BalancedRandomForestClassifier

###### Reference Model

In [169]:
brf_base = BalancedRandomForestClassifier(n_jobs=-1)
brf_base.fit(X_smote, y_smote)

BalancedRandomForestClassifier(n_jobs=-1)

In [170]:
brf_base_pred = brf_base.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_base_pred)

Accuracy: 0.9567790530846485

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      5203
           1       0.81      0.47      0.59       373

    accuracy                           0.96      5576
   macro avg       0.88      0.73      0.78      5576
weighted avg       0.95      0.96      0.95      5576



In [171]:
add_to_tuning_eval_df(
    model=brf_base,
    model_name="BalancedRandomForestClassifier",
    tuning="none",
    best_params="",
    scorer="none",
    x_data_train=X_smote,
    y_data_train=y_smote,
    x_data_test=X_test,
    y_data_test=y_test)

###### Tuning#1a: RandomizedSearchCV

In [172]:
param_grid_brf1 = {
    'n_estimators': [100, 150, 200, 250],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [2, 4, 8, 16, 36, 64],
    'max_leaf_nodes': [9, 11, 13],
}

In [173]:
rs_brf1 = RandomizedSearchCV(BalancedRandomForestClassifier(n_jobs=-1),
                             param_grid_brf1,
                             scoring='f1',
                             verbose=1,
                             n_iter=10)
rs_brf1.fit(X_smote, y_smote)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=BalancedRandomForestClassifier(n_jobs=-1),
                   param_distributions={'max_depth': [2, 4, 8, 16, 36, 64],
                                        'max_features': ['sqrt', 'log2', None],
                                        'max_leaf_nodes': [9, 11, 13],
                                        'n_estimators': [100, 150, 200, 250]},
                   scoring='f1', verbose=1)

In [174]:
print(rs_brf1.best_estimator_)

BalancedRandomForestClassifier(max_depth=64, max_features='log2',
                               max_leaf_nodes=13, n_estimators=150, n_jobs=-1)


In [175]:
brf_tuned1 = BalancedRandomForestClassifier(**rs_brf1.best_params_)
brf_tuned1.fit(X_smote, y_smote)

BalancedRandomForestClassifier(max_depth=64, max_features='log2',
                               max_leaf_nodes=13, n_estimators=150)

In [176]:
brf_tuned1_pred = brf_tuned1.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_tuned1_pred)

Accuracy: 0.9393830703012912

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97      5203
           1       0.56      0.46      0.50       373

    accuracy                           0.94      5576
   macro avg       0.76      0.71      0.73      5576
weighted avg       0.93      0.94      0.94      5576



In [177]:
add_to_tuning_eval_df(
    model=brf_tuned1,
    model_name="BalancedRandomForestClassifier",
    tuning="RandomizedSearchCV",
    scorer="f1",
    best_params=rs_brf1.best_params_,
    x_data_train=X_smote,
    y_data_train=y_smote,
    x_data_test=X_test,
    y_data_test=y_test)

###### Tuning#1b: RandomizedSearchCV (more param)

In [178]:
param_grid_brf2 = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False],
}

In [179]:
rs_brf2 = RandomizedSearchCV(BalancedRandomForestClassifier(n_jobs=-1),
                             param_grid_brf2,
                             scoring='f1',
                             verbose=1,
                             n_iter=10)
rs_brf2.fit(X_smote, y_smote)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=BalancedRandomForestClassifier(n_jobs=-1),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]},
                   scoring='f1', verbose=1)

In [180]:
print(rs_brf2.best_estimator_)

BalancedRandomForestClassifier(bootstrap=True, max_depth=30,
                               max_features='log2', min_samples_leaf=2,
                               n_estimators=200, n_jobs=-1)


In [181]:
brf_tuned2 = BalancedRandomForestClassifier(**rs_brf2.best_params_)
brf_tuned2.fit(X_smote, y_smote)

BalancedRandomForestClassifier(bootstrap=True, max_depth=30,
                               max_features='log2', min_samples_leaf=2,
                               n_estimators=200)

In [182]:
brf_tuned2_pred = brf_tuned2.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_tuned2_pred)

Accuracy: 0.9483500717360115

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      5203
           1       0.67      0.45      0.54       373

    accuracy                           0.95      5576
   macro avg       0.82      0.72      0.75      5576
weighted avg       0.94      0.95      0.94      5576



In [183]:
add_to_tuning_eval_df(
    model=brf_tuned2,
    model_name="BalancedRandomForestClassifier",
    tuning="RandomizedSearchCV",
    scorer="f1",
    best_params=rs_brf2.best_params_,
    x_data_train=X_smote,
    y_data_train=y_smote,
    x_data_test=X_test,
    y_data_test=y_test)

###### Tuning#1c: RandomizedSearchCV (more param) + custom scorer (target f1)

In [184]:
param_grid_brf3 = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False],
}

In [185]:
rs_brf3 = RandomizedSearchCV(BalancedRandomForestClassifier(n_jobs=-1),
                             param_grid_brf3,
                             scoring=custom_scorer_target,
                             verbose=1,
                             n_iter=10)
rs_brf3.fit(X_smote, y_smote)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=BalancedRandomForestClassifier(n_jobs=-1),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]},
                   scoring=make_scorer(custom_f1_target_score, response_method='predict'),
                   verbose=1)

In [186]:
print(rs_brf3.best_estimator_)

BalancedRandomForestClassifier(bootstrap=True, min_samples_leaf=4,
                               min_samples_split=10, n_estimators=500,
                               n_jobs=-1)


In [187]:
brf_tuned3 = BalancedRandomForestClassifier(**rs_brf3.best_params_)
brf_tuned3.fit(X_smote, y_smote)

BalancedRandomForestClassifier(bootstrap=True, min_samples_leaf=4,
                               min_samples_split=10, n_estimators=500)

In [188]:
brf_tuned3_pred = brf_tuned3.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_tuned3_pred)

Accuracy: 0.93525824964132

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97      5203
           1       0.51      0.62      0.56       373

    accuracy                           0.94      5576
   macro avg       0.74      0.79      0.76      5576
weighted avg       0.94      0.94      0.94      5576



In [189]:
add_to_tuning_eval_df(
    model=brf_tuned1,
    model_name="BalancedRandomForestClassifier",
    tuning="RandomizedSearchCV",
    scorer="target",
    best_params=rs_brf1.best_params_,
    x_data_train=X_smote,
    y_data_train=y_smote,
    x_data_test=X_test,
    y_data_test=y_test)

###### Tuning#1d: RandomizedSearchCV (more param) + custom scorer (macro f1)

In [190]:
param_grid_brf4 = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False],
}

In [191]:
rs_brf4 = RandomizedSearchCV(BalancedRandomForestClassifier(n_jobs=-1),
                             param_grid_brf4,
                             scoring=custom_scorer_macro,
                             verbose=1,
                             n_iter=10)
rs_brf4.fit(X_smote, y_smote)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=BalancedRandomForestClassifier(n_jobs=-1),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]},
                   scoring=make_scorer(custom_f1_macro_score, response_method='predict'),
                   verbose=1)

In [192]:
brf_tuned4 = BalancedRandomForestClassifier(**rs_brf4.best_params_)
brf_tuned4.fit(X_smote, y_smote)

BalancedRandomForestClassifier(bootstrap=True, min_samples_leaf=4,
                               min_samples_split=5, n_estimators=500)

In [193]:
brf_tuned4_pred = brf_tuned4.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_tuned4_pred)

Accuracy: 0.936154949784792

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97      5203
           1       0.52      0.61      0.56       373

    accuracy                           0.94      5576
   macro avg       0.75      0.79      0.76      5576
weighted avg       0.94      0.94      0.94      5576



In [194]:
add_to_tuning_eval_df(
    model=brf_tuned4,
    model_name="BalancedRandomForestClassifier",
    tuning="RandomizedSearchCV",
    scorer="f1-macro",
    best_params=rs_brf4.best_params_,
    x_data_train=X_smote,
    y_data_train=y_smote,
    x_data_test=X_test,
    y_data_test=y_test)

###### Tuning#1e: RandomizedSearchCV (more param) + custom scorer (mcc)

In [195]:
param_grid_brf5 = {
'n_estimators': [50, 100, 200, 500],
'max_features': ['sqrt', 'log2'],
'max_depth': [None, 10, 20, 30, 40, 50],
'min_samples_split': [2, 5, 10],
'min_samples_leaf': [1, 2, 4, 6],
'bootstrap': [True, False],
}

In [196]:
rs_brf5 = RandomizedSearchCV(BalancedRandomForestClassifier(n_jobs=-1),
                             param_grid_brf5,
                             scoring=custom_scorer_mcc,
                             verbose=1,
                             n_iter=10)
rs_brf5.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=BalancedRandomForestClassifier(n_jobs=-1),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]},
                   scoring=make_scorer(mcc, response_method='predict'),
                   verbose=1)

In [197]:
brf_tuned5 = BalancedRandomForestClassifier(**rs_brf5.best_params_)
brf_tuned5.fit(X_train, y_train)

BalancedRandomForestClassifier(bootstrap=True, max_features='log2',
                               n_estimators=200)

In [198]:
brf_tuned5_pred = brf_tuned4.predict(X_test)
simple_evaluation(y_test=y_test, y_pred=brf_tuned5_pred)

Accuracy: 0.936154949784792

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97      5203
           1       0.52      0.61      0.56       373

    accuracy                           0.94      5576
   macro avg       0.75      0.79      0.76      5576
weighted avg       0.94      0.94      0.94      5576



In [199]:
add_to_tuning_eval_df(
    model=brf_tuned5,
    model_name="BalancedRandomForestClassifier",
    tuning="RandomizedSearchCV",
    scorer="mcc",
    best_params=rs_brf5.best_params_,
    x_data_train=X_train,
    y_data_train=y_train,
    x_data_test=X_test,
    y_data_test=y_test)

#### Modellvergleich & Test auf finales Testdaten

In [200]:
evaluation_tuning.sort_values(by=["test_f1"], ascending=False)

,model,tuning,scorer,best_params,train_acc,train_prec,train_rec,train_f1,train_mcc,test_acc,test_prec,test_rec,test_f1,test_mcc,clf_rep_train,clf_rep_test
0,BalancedRandomForestClassifier,none,none,,0.999918,1.000000,0.999836,0.999918,0.999836,0.956779,0.805556,0.466488,0.590832,0.593515,precision recall f1-score ...,precision recall f1-score ...
4,BalancedRandomForestClassifier,RandomizedSearchCV,f1-macro,"{'n_estimators': 500, 'min_samples_split': 5, ...",0.975690,0.967200,0.984775,0.975909,0.951536,0.936155,0.519362,0.611260,0.561576,0.529428,precision recall f1-score ...,precision recall f1-score ...
2,BalancedRandomForestClassifier,RandomizedSearchCV,f1,"{'n_estimators': 200, 'min_samples_split': 2, ...",0.961324,0.985444,0.936482,0.960339,0.923790,0.948350,0.670683,0.447721,0.536977,0.522498,precision recall f1-score ...,precision recall f1-score ...
5,BalancedRandomForestClassifier,RandomizedSearchCV,mcc,"{'n_estimators': 200, 'min_samples_split': 2, ...",0.939734,0.502538,1.000000,0.668919,0.685776,0.902439,0.383037,0.750670,0.507246,0.491515,precision recall f1-score ...,precision recall f1-score ...
1,BalancedRandomForestClassifier,RandomizedSearchCV,f1,"{'n_estimators': 150, 'max_leaf_nodes': 13, 'm...",0.929893,0.974178,0.883196,0.926459,0.863560,0.939383,0.557377,0.455764,0.501475,0.472247,precision recall f1-score ...,precision recall f1-score ...
3,BalancedRandomForestClassifier,RandomizedSearchCV,target,"{'n_estimators': 150, 'max_leaf_nodes': 13, 'm...",0.929893,0.974178,0.883196,0.926459,0.863560,0.939383,0.557377,0.455764,0.501475,0.472247,precision recall f1-score ...,precision recall f1-score ...


In [9]:
df_test = pd.read_csv("../../../data/twitter_hate-speech/test_cleaned.csv", index_col=0)
df_test = df_test[df_test.tweet.notna() & df_test.tweet_cleaned.notna()]
df_test.head()

,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
id,,,,,,
4,0,#model i love u take with u all the time in ...,model love take time ur mobile phone kissing f...,0,['#model'],"__mobile_phone__,__kissing_face_with_smiling_e..."
5,0,factsguide: society now #motivation,fact guide society motivation,0,['#motivation'],NaN
6,0,[2/2] huge fan fare and big talking before the...,huge fan big leave chaos pay get show go,0,['#allshowandnogo'],NaN
16,0,ouch...junior is angry😐#got7 #junior #yugyoem ...,junior angry neutral face get junior yo em omg,0,"['#got7', '#junior', '#yugyoem', '#omg']",__neutral_face__
18,1,retweet if you agree!,retweet agree,0,[],NaN


In [22]:
X_test_final = df_test.tweet_cleaned
y_test_final = df_test.label

X_test_final = tfidf_vectorizer.transform(X_test_final)

##### Basismodell BalancedRandomForest

In [23]:
brf_base = BalancedRandomForestClassifier(n_jobs=-1)
brf_base.fit(X_smote, y_smote)

BalancedRandomForestClassifier(n_jobs=-1)

In [26]:
brf_base_pred_final = brf_base.predict(X_test_final)
simple_evaluation(y_test=y_test_final, y_pred=brf_base_pred_final)

Accuracy: 0.9534752094181571

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      8239
           1       0.78      0.43      0.56       595

    accuracy                           0.95      8834
   macro avg       0.87      0.71      0.77      8834
weighted avg       0.95      0.95      0.95      8834



In [27]:
matthews_corrcoef(y_test_final, brf_base_pred_final)

0.5596011895737446